In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import copy
import random
simulateSeconds = 0
intersetions = 0
streets = 0
cars = 0
scoresWhenOnTime = 0
intersetionTable = [[]]
streetToIndex = {}
carsState = []
intersetionLight = {}
streetEndQueue = {}
streetBlockSeconds = {}
prestreetBlockSeconds = {}
maxScore = 0
maxIntersetionLight = {}
inicarsState = []
iniStreetEndQueue = {}
streetCountMap = {}
def readfile(filename):
    global simulateSeconds
    global intersetions
    global streets
    global cars
    global scoresWhenOnTime
    global intersetionTable
    global streetToIndex
    global carsState
    global intersetionLight
    global streetEndQueue
    global iniStreetEndQueue
    global inicarsState
    f = open(filename)
    line = f.readline()
    line = line.replace('\n', '')
    arr = line.split(' ')
    simulateSeconds = int(arr[0])
    intersetions = int(arr[1])
    streets = int(arr[2])
    cars = int(arr[3])
    scoresWhenOnTime = int(arr[4])
    intersetionTable = {}
    carsState = [-1] * cars
    intersectionLightList = {}
    intersetionLight = {}
    streetEndQueue = {}
    for num in range(0, streets):
        line = f.readline()
        line = line.replace('\n', '')
        arr = line.split(' ')
        start = int(arr[0])
        end = int(arr[1])
        streetsName = arr[2]
        timeToDrive = int(arr[3])
        if streetsName == 'bebc-gf':
            print(streetsName + ' need ' + str(timeToDrive))
        streetToIndex[streetsName] = {'start': start, 'end': end}
        if start not in intersetionTable:
            intersetionTable[start] = {}
        intersetionTable[start][end] = {'streetsName': streetsName, 'timeToDrive': timeToDrive}
        
    #print(intersetionLight[1])
    for num in range(0, cars):
        line = f.readline()
        line = line.replace('\n', '')
        arr = line.split(' ')
        streetsNum = int(arr[0])
        streetsArr = arr[1:]
        for streetName in streetsArr:
            if streetName not in streetCountMap:
                streetCountMap[streetName] = 0
            streetCountMap[streetName] = streetCountMap[streetName] + 1
        carsState[num] = {}
        carsState[num]['streetsNum'] = streetsNum
        carsState[num]['arr'] = streetsArr
        carsState[num]['curIndex'] = 0
        carsState[num]['state'] = 'queue'
        carsState[num]['timeConsume'] = 0
        carsState[num]['onRoadSecond'] = 0
        if streetsArr[0] not in streetEndQueue:
            streetEndQueue[streetsArr[0]] = []
        streetEndQueue[streetsArr[0]].append(num)
    inicarsState = copy.deepcopy(carsState)
    iniStreetEndQueue = copy.deepcopy(streetEndQueue)
    for streetName in streetCountMap:
        end = streetToIndex[streetName]['end']
        if end not in intersectionLightList:
            intersectionLightList[end] = []
        intersectionLightList[end].append({'street': streetName, 'count': streetCountMap[streetName]})
    for end in intersectionLightList:
        streets = intersectionLightList[end]
        intersetionLight[end] = []
        random.shuffle(streets)
        for street in streets:
            #for i in range(0, street['count']):
            for i in range(0, 1):
                intersetionLight[end].append(street['street'])

def iniInter():
    global intersetionLight
    f = open('s.txt')
    line = f.readline()
    line = line.replace('\n', '')
    n = int(line)
    intersetionLight = {}
    for i in range(0, n):
        line = f.readline()
        line = line.replace('\n', '')
        end = int(line)
        line = f.readline()
        line = line.replace('\n', '')
        num = int(line)
        intersetionLight[end] = []
        for j in range(0, num):
            line = f.readline()
            line = line.replace('\n', '')
            arr = line.split(' ')
            street = arr[0]
            replaceNum = int(arr[1])
            for k in range(0, replaceNum):
                intersetionLight[end].append(street)

debugcar = -1
def reinit():
    global carsState
    global iniStreetEndQueue
    global inicarsState
    global streetEndQueue
    carsState = copy.deepcopy(inicarsState)
    streetEndQueue = copy.deepcopy(iniStreetEndQueue)
        
        
def simulate():
    global simulateSeconds
    global intersetions
    global streets
    global cars
    global scoresWhenOnTime
    global intersetionTable
    global streetToIndex
    global carsState
    global intersetionLight
    global streetEndQueue
    global streetBlockSeconds
    finishCarNum = 0
    i = -1
    while i < simulateSeconds + 3:
        i = i + 1
        streetQueueToPop = {}
        for j in range(0, cars):
            if j == debugcar:
                print('========second:'+str(i)+'==========')
            if carsState[j]['state'] == 'end':
                continue
            carsState[j]['timeConsume'] = carsState[j]['timeConsume'] + 1
            if carsState[j]['state'] == 'queue':
                streetQueueToPop[carsState[j]['arr'][carsState[j]['curIndex']]] = 1
                
            if carsState[j]['state'] == 'onTheRoad':
                curIndex = carsState[j]['curIndex']
                streetName = carsState[j]['arr'][curIndex]
                streetIndex = streetToIndex[streetName]
                timeToDrive = intersetionTable[streetIndex['start']][streetIndex['end']]['timeToDrive']
                if j== debugcar:
                    print('on the road ' + streetName)
                    print(streetIndex)
                    print('timeToDrive is ' + str(timeToDrive))
                    print(intersetionTable[streetIndex['start']][streetIndex['end']])
                if timeToDrive <= carsState[j]['onRoadSecond']:
                    if curIndex + 1 == len(carsState[j]['arr']):
                        carsState[j]['state'] = 'end'
                        finishCarNum = finishCarNum + 1
                    else:
                        #carsState[j]['curIndex'] = carsState[j]['curIndex'] + 1
                        streetName = carsState[j]['arr'][carsState[j]['curIndex']]
                        carsState[j]['state'] = 'queue'
                        carsState[j]['onRoadSecond'] = 0
                        if streetName not in streetEndQueue:
                            streetEndQueue[streetName] = []
                        streetEndQueue[streetName].append(j)
                        streetQueueToPop[carsState[j]['arr'][carsState[j]['curIndex']]] = 1
                        if j == debugcar:
                            print(streetQueueToPop)
                else:
                    carsState[j]['onRoadSecond'] = carsState[j]['onRoadSecond'] + 1
                    if carsState[j]['onRoadSecond'] == timeToDrive and curIndex + 1 == len(carsState[j]['arr']):
                        carsState[j]['state'] = 'end'
                        finishCarNum = finishCarNum + 1
                        continue;
                        
          
        for street in streetQueueToPop:
            end = streetToIndex[street]['end']
            lightStreet = intersetionLight[end]
            totalWait = len(lightStreet)
            steetToGreenIndex = i%totalWait;
            #print(intersetionLight[end][steetToGreenIndex] + ' is green light when street ' + street + ' is waiting')
            if intersetionLight[end][steetToGreenIndex] == street:
                carPassLightIndex = streetEndQueue[street].pop(0)
                if len(streetEndQueue[street])>0:
                    if street not in streetBlockSeconds:
                        streetBlockSeconds[street] = 0
                    streetBlockSeconds[street] = streetBlockSeconds[street] + len(streetEndQueue[street])
                carsState[carPassLightIndex]['state'] = 'onTheRoad'
                carsState[carPassLightIndex]['curIndex'] = carsState[carPassLightIndex]['curIndex'] + 1
                carsState[carPassLightIndex]['onRoadSecond'] = 1
                curIndex = carsState[carPassLightIndex]['curIndex']
                streetName = carsState[carPassLightIndex]['arr'][curIndex]
                streetIndex = streetToIndex[streetName]
                timeToDrive = intersetionTable[streetIndex['start']][streetIndex['end']]['timeToDrive']
                if carsState[carPassLightIndex]['curIndex'] + 1 == len(carsState[carPassLightIndex]['arr']) and carsState[carPassLightIndex]['onRoadSecond'] == timeToDrive:
                    carsState[carPassLightIndex]['state'] = 'end'
                if carPassLightIndex == debugcar:
                    print('car ' + str(carPassLightIndex) + ' pass ' + street )
            else:
                if street not in streetBlockSeconds:
                    streetBlockSeconds[street] = 0
                streetBlockSeconds[street] = streetBlockSeconds[street] + len(streetEndQueue[street])
        for j in range(0, cars):
            if j == debugcar:
                print('car:'+ str(j))
                print('state:' + carsState[j]['state'])
                print('curIndex: ' + str(carsState[j]['curIndex']))
                print('len:' + str(len(carsState[j]['arr'])))   
                print('onRoadSecond:' + str(carsState[j]['onRoadSecond']))
                print('timeConsume:' + str(carsState[j]['timeConsume']))
                curIndex = carsState[j]['curIndex']
                if curIndex < len(carsState[j]['arr']): 
                    streetName = carsState[j]['arr'][curIndex]
                    streetIndex = streetToIndex[streetName]
                    print('pass street ' + streetName + " need " + str(intersetionTable[streetIndex['start']][streetIndex['end']]['timeToDrive']) )
                print('roads:')
                print(carsState[j]['arr'])
                
                
def calScore():
    global simulateSeconds
    global carsState
    global scoresWhenOnTime
    global maxScore
    global intersetionLight
    global maxIntersetionLight
    global streetBlockSeconds
    global prestreetBlockSeconds
    global streetToIndex
    score = 0
    for car in carsState:
        if car['timeConsume']>simulateSeconds:
            continue
        score = score + simulateSeconds - car['timeConsume'] + scoresWhenOnTime
    print('score: ' + str(score))
    if maxScore == 0:
        prestreetBlockSeconds = copy.deepcopy(streetBlockSeconds)
        maxIntersetionLight = copy.deepcopy(intersetionLight)
        print('ini')
    if score >= maxScore:
        maxScore = score
        print('cover')
        maxIntersetionLight = copy.deepcopy(intersetionLight)
            
            
    intersetionLight = copy.deepcopy(maxIntersetionLight)
    #reinit()
    #simulate() 
    #score = 0    
    #for car in carsState:
    #    if car['timeConsume']>simulateSeconds:
    #        continue
    #    score = score + simulateSeconds - car['timeConsume'] + scoresWhenOnTime
    #print('after: ' + str(score))

def optimal():
    global streetBlockSeconds
    global streetToIndex
    global intersetionLight
    blockStreet = []
    modifyInter = {}
    for street in streetBlockSeconds:
        #print(street + ' block ' + str(streetBlockSeconds[street]) + ' seconds');
        for i in range(0, streetBlockSeconds[street]):
            blockStreet.append(street)
    length = len(blockStreet)
    for i in range(0, int(length/1000)):
        index = random.randint(0, length-1)
        streetName = blockStreet[index]
        modifyInter[streetName] = 1
    #print(modifyInter)
    for streetName in modifyInter:
        streetIndex = streetToIndex[streetName]
        intersetion = streetIndex['end']
        newList = []
        streetMap = {}
        streetList = intersetionLight[intersetion]
        for street in streetList:
            if street not in streetMap:
                streetMap[street] = 0
            streetMap[street] = streetMap[street] + 1
        for street in streetMap:
            newList.append({'street': street, 'count': 1})
            #newList.append({'street': street, 'count': streetMap[street]})
        index = random.randint(0, len(newList) - 1)
        #if newList[index]['street'] == streetName:
        #   newList[index]['count'] = newList[index]['count'] + 1
        #elif newList[index]['count']>1:
            #newList[index]['count'] = newList[index]['count'] -1
        random.shuffle(newList)
        intersetionLight[intersetion] = []
        for item in newList:
            for i in range(0, item['count']):
                intersetionLight[intersetion].append(item['street'])
        
def writeFile():
    f = open('submission.csv', 'w')
    lenght = len(maxIntersetionLight.keys())
    f.write(str(lenght)+'\n')
    for key in maxIntersetionLight:
        f.write(str(key)+'\n')
        streetMap = {}
        streetList = maxIntersetionLight[key]
        
        for street in streetList:
            if street not in streetMap:
                streetMap[street] = 0
            streetMap[street] = streetMap[street] + 1
        f.write(str(len(streetMap.keys()))+'\n')
        streetList = sorted(set(streetList),key=streetList.index)
        for street in streetList:
            f.write(street + ' ' + str(streetMap[street]) + '\n')
readfile('/kaggle/input/hashcode-2021-oqr-extension/hashcode.in')
#readfile('hashcode.in')
for i in range(0, 30):
    reinit()
    #iniInter()
    simulate()  
    calScore()
    writeFile()
    optimal()
